In [2]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# Sample Data
data = {
    "Transaction ID": [1001, 1002, 1003, 1004, 1005],
    "Product Name": ["Laptop", "Smartphone", "Headphones", "Tablet", "Monitor"],
    "Quantity Sold": [2, 5, 3, 1, 2],
    "Unit Price": [800, 500, 150, 400, 300],
    "Total Sales": [1600, 2500, 450, 400, 600],
    "Date of Sale": ["8/1/2025", "8/1/2025", "8/2/2025", "8/3/2025", "8/4/2025"]
}

# Convert Data into DataFrame
df = pd.DataFrame(data)
df['Date of Sale'] = pd.to_datetime(df['Date of Sale'])

# Calculate the total sales by product for pie chart
sales_by_product = df.groupby('Product Name')['Total Sales'].sum().reset_index()

# Initialize Dash App
app = Dash(__name__)

# Layout of the app
app.layout = html.Div([
    html.H1("Interactive Sales Dashboard"),
    
    # Dropdown to select product
    dcc.Dropdown(
        id='product-dropdown',
        options=[{'label': product, 'value': product} for product in df['Product Name'].unique()],
        value='Laptop',  # Default selected value
        multi=False
    ),
    
    # Graph for Total Sales
    dcc.Graph(id='sales-graph'),
    
    # Graph for Quantity Sold
    dcc.Graph(id='quantity-graph'),
    
    # Pie chart for Total Sales Distribution by Product
    dcc.Graph(id='pie-chart')
])

# Callback to update the graphs based on selected product
@app.callback(
    [Output('sales-graph', 'figure'),
     Output('quantity-graph', 'figure'),
     Output('pie-chart', 'figure')],
    [Input('product-dropdown', 'value')]
)
def update_graph(selected_product):
    # Filter data based on selected product
    filtered_df = df[df['Product Name'] == selected_product]
    
    # Create the Total Sales bar graph
    sales_fig = px.bar(filtered_df, 
                       x='Date of Sale', 
                       y='Total Sales', 
                       title=f"Total Sales for {selected_product}",
                       labels={"Total Sales": "Sales ($)", "Date of Sale": "Date"})
    
    # Create the Quantity Sold bar graph
    quantity_fig = px.bar(filtered_df, 
                          x='Date of Sale', 
                          y='Quantity Sold', 
                          title=f"Quantity Sold for {selected_product}",
                          labels={"Quantity Sold": "Quantity Sold", "Date of Sale": "Date"},
                          color_discrete_sequence=["#EF553B"]
                         )
    
    # Create the Pie chart for Total Sales Distribution by Product
    pie_fig = px.pie(sales_by_product, 
                     names='Product Name', 
                     values='Total Sales', 
                     title="Total Sales Distribution by Product")
    
    return sales_fig, quantity_fig, pie_fig

# Run the app in Jupyter notebook
app.run(mode="inline")
